In [1]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf

from tensorflow import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM, Reshape, RepeatVector, TimeDistributed
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from collections import Counter

import numpy as np
import os
from mido import MidiFile, MidiTrack, Message
import mido

from scipy.special import softmax

Using TensorFlow backend.


In [2]:
paths = []
songs = []
for r, d, f in os.walk(r"C:\Users\Vee\Desktop\python\blues"):
    for file in f:
        if '.mid' in file:
            paths.append(os.path.join(r, file))

for path in paths:
    mid = MidiFile(path, type = 1)
    songs.append(mid)

In [3]:
notes = []
dataset = []

In [4]:
#for each in midi object in list of songs
for i in range(len(songs)):
    #for each note in midi object
    for msg in songs[i]:
        #filtering out meta messages
        if not msg.is_meta:
            #filtering out control changes
            if (msg.type == 'note_on'):
                #normalizing note and velocity values
                notes.append([msg.note/127])
    #if more than 30 notes delete them
    if len(notes) > 40:
        for x in range(len(notes)-40):
            notes = np.delete(notes, 40, 0)
    #if less than 30 notes pad with zeros
    elif len(notes) < 40:
        for x in range(40 - len(notes)):
            notes.append([0,0,0])
    dataset.append(notes)    
    notes = []

In [5]:
dataset = np.array(dataset)
dataset.shape

(20, 40, 1)

In [6]:
bruv = []
#for each in midi object in list of songs
for i in range(len(songs)):
    #for each note in midi object
    for msg in songs[i]:
        #filtering out meta messages
        if not msg.is_meta:
            #filtering out control changes
            if (msg.type == 'note_on'):
                #normalizing note and velocity values
                notes.append([msg.note/127])
    #if more than 40 notes delete them
    if len(notes) > 4:
        for x in range(len(notes)-4):
            notes = np.delete(notes, 4, 0)
    bruv.append(notes)    
    notes = []

In [7]:
bruv = np.array(bruv)
bruv.shape

(20, 4, 1)

In [8]:
#splitting data into train and test sets. 3/4 train, 1/4 test.
x_train,x_test,y_train,y_test = train_test_split(bruv, dataset, test_size=0.2, shuffle=False, random_state=42)

In [16]:
# define model
model = Sequential()
#shaping input to match data
model.add(LSTM(600, activation='relu', input_shape=(4, 1)))
#specifying output to have 60 timesteps
model.add(RepeatVector(40))
model.add(LSTM(600, activation='tanh', return_sequences=True))
model.add(LSTM(600, activation='sigmoid', return_sequences=True))
model.add(LSTM(600, activation='selu', return_sequences=True))
#specifying 3 features as the output
model.add(TimeDistributed(Dense(1024, activation='tanh')))
model.add(TimeDistributed(Dense(1024)))
model.add(TimeDistributed(Dense(1024)))
model.add(TimeDistributed(Dense(1024)))
model.add(TimeDistributed(Dense(512)))
model.add(TimeDistributed(Dense(1)))
model.compile(loss='mean_absolute_error', optimizer='adam')
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 600)               1444800   
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 40, 600)           0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 40, 600)           2882400   
_________________________________________________________________
lstm_7 (LSTM)                (None, 40, 600)           2882400   
_________________________________________________________________
lstm_8 (LSTM)                (None, 40, 600)           2882400   
_________________________________________________________________
time_distributed_4 (TimeDist (None, 40, 1024)          615424    
_________________________________________________________________
time_distributed_5 (TimeDist (None, 40, 1024)         

In [17]:
model.fit(bruv, dataset, epochs=50, batch_size=10, verbose=2,validation_data=(x_test, y_test))

Train on 20 samples, validate on 4 samples
Epoch 1/50
 - 5s - loss: 17.9743 - val_loss: 42.0334
Epoch 2/50
 - 1s - loss: 25.7410 - val_loss: 4.4664
Epoch 3/50
 - 1s - loss: 4.6003 - val_loss: 5.2779
Epoch 4/50
 - 1s - loss: 6.3773 - val_loss: 4.4485
Epoch 5/50
 - 1s - loss: 3.8233 - val_loss: 2.6772
Epoch 6/50
 - 1s - loss: 3.6335 - val_loss: 3.5252
Epoch 7/50
 - 1s - loss: 4.2538 - val_loss: 3.4217
Epoch 8/50
 - 1s - loss: 2.9750 - val_loss: 2.3543
Epoch 9/50
 - 1s - loss: 3.2081 - val_loss: 3.3493
Epoch 10/50
 - 1s - loss: 3.6604 - val_loss: 2.9330
Epoch 11/50
 - 1s - loss: 2.9925 - val_loss: 0.6819
Epoch 12/50
 - 1s - loss: 1.0807 - val_loss: 3.4185
Epoch 13/50
 - 1s - loss: 2.4544 - val_loss: 4.4270
Epoch 14/50
 - 1s - loss: 3.7366 - val_loss: 1.9544
Epoch 15/50
 - 1s - loss: 2.5036 - val_loss: 1.1825
Epoch 16/50
 - 1s - loss: 2.5026 - val_loss: 3.7654
Epoch 17/50
 - 1s - loss: 2.4977 - val_loss: 1.1646
Epoch 18/50
 - 1s - loss: 1.0938 - val_loss: 0.7630
Epoch 19/50
 - 1s - loss: 1

In [18]:
fuckie = [[bruv[8][0]+1, bruv[8][1]-1, bruv[8][2]+1, bruv[8][3]-1]]
fuckie = np.array(fuckie)
fuckie.reshape(1,4,1)
predict = model.predict(bruv)
#disregarding negative values
predict = abs(predict)

jamie = model.predict_on_batch(fuckie)

In [19]:
#adjusting from normalization
for y in range(len(songs)):
    for x in range(40):
            predict[y][x][0] = predict[y][x][0] * 127
            while(predict[y][x][0] > 127):
                predict[y][x][0] = predict[y][x][0] / 10
            print(predict[y][x])

[62.021385]
[72.74055]
[65.14463]
[64.08555]
[65.42118]
[65.73144]
[66.09309]
[64.80376]
[64.11335]
[67.05626]
[71.664116]
[73.615]
[73.43909]
[71.43021]
[70.53438]
[68.71323]
[70.84828]
[70.838104]
[70.84828]
[70.84828]
[70.53103]
[69.185104]
[70.192085]
[68.85632]
[69.09701]
[72.08656]
[69.420296]
[72.47691]
[69.50104]
[70.32486]
[72.23358]
[69.84692]
[70.18334]
[69.10802]
[69.83851]
[69.83851]
[69.83851]
[68.691864]
[69.52725]
[69.73295]
[62.021717]
[72.74164]
[65.146736]
[64.08482]
[65.42183]
[65.729774]
[66.09356]
[64.80491]
[64.116]
[67.05936]
[71.66411]
[73.61491]
[73.43914]
[71.43007]
[70.53438]
[68.71487]
[70.84828]
[70.838104]
[70.84828]
[70.84828]
[70.53103]
[69.185104]
[70.19209]
[68.85632]
[69.09703]
[72.08656]
[69.420296]
[72.24206]
[69.50104]
[70.32486]
[72.23358]
[69.84692]
[70.18334]
[69.10802]
[69.83851]
[69.83851]
[69.83851]
[68.691864]
[69.52725]
[69.73295]
[62.02582]
[72.7568]
[65.17418]
[64.07441]
[65.42943]
[65.709694]
[66.102806]
[64.8215]
[64.15253]
[67.10778]


In [20]:
#forced mode
mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)

track.append(Message('program_change', program=1, time=0))
x = 3
for g in range(5):
    for i in range(40):
        track.append(Message('note_on', note= int(predict[x][i][0]), velocity=64, time=77)) #how much of space to use
        track.append(Message('note_off', note= int(predict[x][i][0]), velocity=64, time=120)) #allocate space
mid.save('james.mid')

In [66]:
#adjusting from normalization
for x in range(40):
    jamie[0][x][0] = abs(jamie[0][x][0] * 65)
    while(jamie[0][x][0] > 127):
        jamie[0][x][0] = jamie[0][x][0] / 10

In [67]:
jamie
#jamie[0][0][0]

array([[[28.179256],
        [31.832243],
        [35.73044 ],
        [40.07002 ],
        [41.683033],
        [37.100285],
        [27.54219 ],
        [18.623947],
        [15.705645],
        [20.647528],
        [30.149506],
        [36.90457 ],
        [36.52391 ],
        [30.026783],
        [22.288431],
        [18.37427 ],
        [21.339382],
        [29.241508],
        [36.59851 ],
        [38.465096],
        [34.578613],
        [28.392176],
        [24.612453],
        [25.809368],
        [30.31061 ],
        [34.412754],
        [35.253483],
        [32.668613],
        [28.814014],
        [26.455925],
        [26.81446 ],
        [29.11239 ],
        [31.362503],
        [31.981522],
        [30.828722],
        [29.02135 ],
        [27.911886],
        [28.08576 ],
        [29.216202],
        [30.409018]]], dtype=float32)

In [68]:
#free mode
mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)

track.append(Message('program_change', program=1, time=0))
for x in range(5):
    for i in range(39):
        track.append(Message('note_on', note= int(jamie[0][i][0]), velocity=64, time=77))
        track.append(Message('note_off', note= int(jamie[0][i][0]), velocity=64, time=120))
mid.save('jamie.mid')

In [14]:
from keras.models import load_model

# Creates a HDF5 file 'my_model.h5'
model.save('my_model.h5')